**Dependencies**

In [ ]:
# Installing python dependencies
%pip install openai jsonlines

In [2]:
# Import Libraries

import json
from openai import OpenAI

**Module**

In [ ]:
%pip install nbformat

In [ ]:
%pip install pandas

**Authentication**

In [5]:
# Initializing GPT Api
api_key = ""
client = OpenAI(api_key=api_key)

**Dataset**

In [ ]:
#Trip dataset
data = [
#Penang data
#1 Day
{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Pak Mat Western Cafe, Remarks: Kickstart your day at Pak Mat Western Cafe, where a hearty breakfast awaits. This cozy spot offers a delightful mix of Western flavors, setting the perfect tone for a day of exploration in Penang., Link: https://maps.app.goo.gl/CQL1DLg7X4shKgqJ7} {Event: Lunch, Place: BM Famous Ban Chien Kueh, Remarks: Elevenses at BM Famous Ban Chien Kueh is a must! Indulge in the delightful taste of this local favorite, a perfect mid-morning treat that captures the essence of Penang's culinary traditions., Link: https://maps.app.goo.gl/kTBVVoMQKaL92Gjz7} {Event: Elevenses, Place: Aki Pancake, Remarks: Elevenses take a delightful turn at Aki Pancake. Indulge in a variety of fluffy pancakes with a twist, making it a perfect spot to satisfy your mid-morning cravings., Link: https://maps.app.goo.gl/M8qxkWgBsvZnitRG9} {Event: Dinner, Place: Queensbay Mall, Remarks: Queensbay Mall is not just a shopping destination; it's a complete experience. Enjoy a delicious dinner and indulge in some retail therapy, surrounded by a vibrant atmosphere and a plethora of choices., Link: https://maps.app.goo.gl/idTHc6p8gqW2AiTaA} {Event: Attractions, Place: Mengkuang Dam, Remarks: Discover the tranquility of nature at Mengkuang Dam. The serene surroundings and picturesque landscapes make it an ideal destination for those seeking a peaceful escape and scenic beauty., Link: https://maps.app.goo.gl/E3w4QwVDAqQTxT2EA} {Event: Check in hotel, Place: Tropicana 218 Macalister, Remarks: Experience comfort and style at Tropicana 218 Macalister. Your home away from home, this hotel provides a welcoming retreat, ensuring a relaxing stay in the heart of Penang., Link: https://maps.app.goo.gl/Mi3tnvAdj62bJXvL9} {Event: Attractions, Place: Pantai Seagate View Pulau Jerjak, Remarks: Explore the breathtaking views at Pantai Seagate on Pulau Jerjak. This hidden gem offers a unique perspective of Penang's coastal beauty, making it a must-visit destination for nature enthusiasts., Link: https://maps.app.goo.gl/a9n8aBvHB3ezAARZ6} {Event: Dessert, Place: Bingsu Monster, Remarks: Satisfy your sweet cravings at Bingsu Monster. This dessert haven offers a cool and refreshing escape with its delightful array of Bingsu creations, a perfect way to beat the Penang heat., Link: https://maps.app.goo.gl/PZB6RouxCKmrxi15A}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Pak Mat Western Cafe, Remarks: Kickstart your day at Pak Mat Western Cafe, where a hearty breakfast awaits. This cozy spot offers a delightful mix of Western flavors, setting the perfect tone for a day of exploration in Penang., Link: https://maps.app.goo.gl/CQL1DLg7X4shKgqJ7} {Event: Lunch, Place: BM Famous Ban Chien Kueh, Remarks: Elevenses at BM Famous Ban Chien Kueh is a must! Indulge in the delightful taste of this local favorite, a perfect mid-morning treat that captures the essence of Penang's culinary traditions., Link: https://maps.app.goo.gl/kTBVVoMQKaL92Gjz7} {Event: Elevenses, Place: Aki Pancake, Remarks: Elevenses take a delightful turn at Aki Pancake. Indulge in a variety of fluffy pancakes with a twist, making it a perfect spot to satisfy your mid-morning cravings., Link: https://maps.app.goo.gl/M8qxkWgBsvZnitRG9} {Event: Dinner, Place: Queensbay Mall, Remarks: Queensbay Mall is not just a shopping destination; it's a complete experience. Enjoy a delicious dinner and indulge in some retail therapy, surrounded by a vibrant atmosphere and a plethora of choices., Link: https://maps.app.goo.gl/idTHc6p8gqW2AiTaA} {Event: Dessert, Place: Bingsu Monster, Remarks: Satisfy your sweet cravings at Bingsu Monster. This dessert haven offers a cool and refreshing escape with its delightful array of Bingsu creations, a perfect way to beat the Penang heat., Link: https://maps.app.goo.gl/PZB6RouxCKmrxi15A} {Event: Check in hotel, Place: Tropicana 218 Macalister, Remarks: Experience comfort and style at Tropicana 218 Macalister. Your home away from home, this hotel provides a welcoming retreat, ensuring a relaxing stay in the heart of Penang., Link: https://maps.app.goo.gl/Mi3tnvAdj62bJXvL9} {Event: Attractions, Place: Pantai Seagate View Pulau Jerjak, Remarks: Explore the breathtaking views at Pantai Seagate on Pulau Jerjak. This hidden gem offers a unique perspective of Penang's coastal beauty, making it a must-visit destination for nature enthusiasts., Link: https://maps.app.goo.gl/a9n8aBvHB3ezAARZ6} {Event: Attractions, Place: Mengkuang Dam, Remarks: Discover the tranquility of nature at Mengkuang Dam. The serene surroundings and picturesque landscapes make it an ideal destination for those seeking a peaceful escape and scenic beauty., Link: https://maps.app.goo.gl/E3w4QwVDAqQTxT2EA}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Pak Mat Western Cafe, Remarks: Kickstart your day at Pak Mat Western Cafe, where a hearty breakfast awaits. This cozy spot offers a delightful mix of Western flavors, setting the perfect tone for a day of exploration in Penang., Link: https://maps.app.goo.gl/CQL1DLg7X4shKgqJ7} {Event: Lunch, Place: BM Famous Ban Chien Kueh, Remarks: Elevenses at BM Famous Ban Chien Kueh is a must! Indulge in the delightful taste of this local favorite, a perfect mid-morning treat that captures the essence of Penang's culinary traditions., Link: https://maps.app.goo.gl/kTBVVoMQKaL92Gjz7} {Event: Attractions, Place: Mengkuang Dam, Remarks: Discover the tranquility of nature at Mengkuang Dam. The serene surroundings and picturesque landscapes make it an ideal destination for those seeking a peaceful escape and scenic beauty., Link: https://maps.app.goo.gl/E3w4QwVDAqQTxT2EA} {Event: Dinner, Place: Queensbay Mall, Remarks: Queensbay Mall is not just a shopping destination; it's a complete experience. Enjoy a delicious dinner and indulge in some retail therapy, surrounded by a vibrant atmosphere and a plethora of choices., Link: https://maps.app.goo.gl/idTHc6p8gqW2AiTaA} {Event: Dessert, Place: Bingsu Monster, Remarks: Satisfy your sweet cravings at Bingsu Monster. This dessert haven offers a cool and refreshing escape with its delightful array of Bingsu creations, a perfect way to beat the Penang heat., Link: https://maps.app.goo.gl/PZB6RouxCKmrxi15A} {Event: Check in hotel, Place: Tropicana 218 Macalister, Remarks: Experience comfort and style at Tropicana 218 Macalister. Your home away from home, this hotel provides a welcoming retreat, ensuring a relaxing stay in the heart of Penang., Link: https://maps.app.goo.gl/Mi3tnvAdj62bJXvL9} {Event: Elevenses, Place: Aki Pancake, Remarks: Elevenses take a delightful turn at Aki Pancake. Indulge in a variety of fluffy pancakes with a twist, making it a perfect spot to satisfy your mid-morning cravings., Link: https://maps.app.goo.gl/M8qxkWgBsvZnitRG9} {Event: Attractions, Place: Pantai Seagate View Pulau Jerjak, Remarks: Explore the breathtaking views at Pantai Seagate on Pulau Jerjak. This hidden gem offers a unique perspective of Penang's coastal beauty, making it a must-visit destination for nature enthusiasts., Link: https://maps.app.goo.gl/a9n8aBvHB3ezAARZ6}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Pak Mat Western Cafe, Remarks: Kickstart your day at Pak Mat Western Cafe, where a hearty breakfast awaits. This cozy spot offers a delightful mix of Western flavors, setting the perfect tone for a day of exploration in Penang., Link: https://maps.app.goo.gl/CQL1DLg7X4shKgqJ7} {Event: Lunch, Place: BM Famous Ban Chien Kueh, Remarks: Elevenses at BM Famous Ban Chien Kueh is a must! Indulge in the delightful taste of this local favorite, a perfect mid-morning treat that captures the essence of Penang's culinary traditions., Link: https://maps.app.goo.gl/kTBVVoMQKaL92Gjz7} {Event: Attractions, Place: Pantai Seagate View Pulau Jerjak, Remarks: Explore the breathtaking views at Pantai Seagate on Pulau Jerjak. This hidden gem offers a unique perspective of Penang's coastal beauty, making it a must-visit destination for nature enthusiasts., Link: https://maps.app.goo.gl/a9n8aBvHB3ezAARZ6} {Event: Dinner, Place: Queensbay Mall, Remarks: Queensbay Mall is not just a shopping destination; it's a complete experience. Enjoy a delicious dinner and indulge in some retail therapy, surrounded by a vibrant atmosphere and a plethora of choices., Link: https://maps.app.goo.gl/idTHc6p8gqW2AiTaA} {Event: Attractions, Place: Mengkuang Dam, Remarks: Discover the tranquility of nature at Mengkuang Dam. The serene surroundings and picturesque landscapes make it an ideal destination for those seeking a peaceful escape and scenic beauty., Link: https://maps.app.goo.gl/E3w4QwVDAqQTxT2EA} {Event: Check in hotel, Place: Tropicana 218 Macalister, Remarks: Experience comfort and style at Tropicana 218 Macalister. Your home away from home, this hotel provides a welcoming retreat, ensuring a relaxing stay in the heart of Penang., Link: https://maps.app.goo.gl/Mi3tnvAdj62bJXvL9} {Event: Dessert, Place: Bingsu Monster, Remarks: Satisfy your sweet cravings at Bingsu Monster. This dessert haven offers a cool and refreshing escape with its delightful array of Bingsu creations, a perfect way to beat the Penang heat., Link: https://maps.app.goo.gl/PZB6RouxCKmrxi15A} {Event: Elevenses, Place: Aki Pancake, Remarks: Elevenses take a delightful turn at Aki Pancake. Indulge in a variety of fluffy pancakes with a twist, making it a perfect spot to satisfy your mid-morning cravings., Link: https://maps.app.goo.gl/M8qxkWgBsvZnitRG9}"
        }
    ]
},

{
  "messages": [
    {
      "role": "system",
      "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
    },
    {
      "role": "user",
      "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
    },
    {
      "role": "assistant",
      "content": "Day 1: {Event: Breakfast, Place: Toh Soon Cafe, Remarks: Begin your day at Toh Soon Cafe, a charming spot offering traditional breakfast delights. The nostalgic atmosphere and local flavors set the stage for a day of exploring George Town., Link: https://maps.app.goo.gl/3rZqwWU9FE9u9X8Y6} {Event: Street Art Trail, Place: Armenian Street, Remarks: Embark on a street art adventure along Armenian Street. Marvel at the vibrant murals that tell the story of George Town's rich history and culture., Link: https://maps.app.goo.gl/NnWNSA1qDTB9csxz8} {Event: Lunch, Place: Tek Sen Restaurant, Remarks: Enjoy a delightful lunch at Tek Sen Restaurant, known for its authentic Nyonya and Chinese cuisine. The diverse menu caters to various tastes, providing a true culinary experience., Link: https://maps.app.goo.gl/9AAVrrX7VL4oPVuWA} {Event: Afternoon Tea, Place: China House, Remarks: Unwind with afternoon tea at China House, a unique venue blending art, food, and coffee. Indulge in sweet and savory treats in a relaxed and artistic setting., Link: https://maps.app.goo.gl/Y4hRmyDChNoS1U1C6} {Event: Explore Heritage Buildings, Place: Khoo Kongsi, Remarks: Immerse yourself in the architectural beauty of Khoo Kongsi. This historic clan house showcases intricate details and provides insight into Penang's heritage., Link: https://maps.app.goo.gl/Fh3nA7DEMWmPbG4n7} {Event: Dinner, Place: Red Garden Food Paradise, Remarks: Conclude your day with dinner at Red Garden Food Paradise. A hawker haven, it offers a variety of local and international dishes amidst a lively atmosphere., Link: https://maps.app.goo.gl/2mpw8jsP4pA3D9SKA} {Event: Night Market, Place: Batu Ferringhi Night Market, Remarks: Experience the vibrant Batu Ferringhi Night Market. Stroll through the stalls, shop for souvenirs, and soak in the lively atmosphere by the beach., Link: https://maps.app.goo.gl/MoYGJSJ9NrBMnQVz7} {Event: Relaxing Evening, Place: Straits Quay, Remarks: Unwind at Straits Quay, a waterfront mall offering a relaxing evening by the sea. Enjoy a leisurely stroll, shop, or dine with a beautiful view., Link: https://maps.app.goo.gl/EzBYV8CQHqdC86dz7}"
    }
  ]
},

{
  "messages": [
    {
      "role": "system",
      "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
    },
    {
      "role": "user",
      "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
    },
    {
      "role": "assistant",
      "content": "Day 1: {Event: Breakfast, Place: China House, Remarks: Begin your day at China House, a unique venue blending art, food, and coffee. Kickstart your morning with a diverse breakfast selection in a creative and vibrant setting., Link: https://maps.app.goo.gl/Y4hRmyDChNoS1U1C6} {Event: Penang National Park, Place: Canopy Walkway, Remarks: Immerse yourself in nature with a visit to Penang National Park. Traverse the Canopy Walkway for a unique treetop experience, surrounded by lush greenery., Link: https://maps.app.goo.gl/p1jRrCw5F5v3CffU7} {Event: Beach Time, Place: Turtle Beach, Remarks: Relax at Turtle Beach, a serene and less crowded spot within Penang National Park. Enjoy the sun, sand, and sea in a tranquil environment., Link: https://maps.app.goo.gl/dGmJv1h3U45x7aWQ6} {Event: Lunch, Place: Floating Seafood Restaurant, Remarks: Indulge in a seafood feast at the Floating Seafood Restaurant. Located near Batu Uban, it offers a unique dining experience on the water., Link: https://maps.app.goo.gl/GorTiLz6NFoq58Ws8} {Event: Monkey Beach, Place: Monkey Beach, Remarks: Explore Monkey Beach, a picturesque destination accessible by boat or a hike. Enjoy the natural beauty and encounter playful monkeys along the shoreline., Link: https://maps.app.goo.gl/Efe1dABoAX1gM8fN7} {Event: Dinner, Place: The Spice Market, Remarks: Conclude your day with dinner at The Spice Market. This restaurant offers a blend of flavors inspired by the spice route, providing a culinary journey., Link: https://maps.app.goo.gl/srr9atNZjvWU2HfF9} {Event: Night Adventure, Place: Entopia by Penang Butterfly Farm, Remarks: Embark on a night adventure at Entopia. Explore the magical world of butterflies and insects in their nocturnal habitat., Link: https://maps.app.goo.gl/uq3dfu68sSxQ8TwS6} {Event: Relaxing Evening, Place: Straits Quay, Remarks: Unwind at Straits Quay, a waterfront mall offering a relaxing evening by the sea. Enjoy a leisurely stroll, shop, or dine with a beautiful view., Link: https://maps.app.goo.gl/EzBYV8CQHqdC86dz7}"
    }
  ]
},

{
  "messages": [
    {
      "role": "system",
      "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
    },
    {
      "role": "user",
      "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
    },
    {
      "role": "assistant",
      "content": "Day 1: {Event: Breakfast, Place: Black Kettle, Remarks: Start your day at Black Kettle, a cozy cafe offering a delightful breakfast menu. Enjoy artisanal coffee and a variety of morning treats in a relaxed setting., Link: https://maps.app.goo.gl/sQ22WXq7rJgv9hbo7} {Event: Penang Hill, Place: The Habitat Penang Hill, Remarks: Explore Penang Hill and visit The Habitat. Take a walk through the treetops on the Langur Way Canopy Walk and enjoy panoramic views of the island., Link: https://maps.app.goo.gl/C1BWwHQr4MQzrXWt9} {Event: Lunch, Place: Tek Sen Restaurant, Remarks: Indulge in lunch at Tek Sen Restaurant, known for its authentic Nyonya and Chinese cuisine. The diverse menu caters to various tastes, providing a true culinary experience., Link: https://maps.app.goo.gl/9AAVrrX7VL4oPVuWA} {Event: Peranakan Mansion, Place: Pinang Peranakan Mansion, Remarks: Immerse yourself in the Peranakan culture at Pinang Peranakan Mansion. Explore the rich heritage and opulent lifestyle of the Straits Chinese community., Link: https://maps.app.goo.gl/3ft4qPUBS91JuTAA9} {Event: Dinner, Place: The Spice Market, Remarks: Conclude your day with dinner at The Spice Market. This restaurant offers a blend of flavors inspired by the spice route, providing a culinary journey., Link: https://maps.app.goo.gl/srr9atNZjvWU2HfF9} {Event: Street Food Adventure, Place: Gurney Drive Hawker Centre, Remarks: Experience a street food adventure at Gurney Drive Hawker Centre. Indulge in a variety of local delights amidst the lively atmosphere., Link: https://maps.app.goo.gl/v2FLdzmXrB4msoNf8} {Event: Nightlife, Place: The Safe Room, Remarks: Explore the nightlife at The Safe Room, a speakeasy-style bar offering unique cocktails. Enjoy a sophisticated and hidden gem in the heart of George Town., Link: https://maps.app.goo.gl/8R4jTPGcUw1UFCs2A} {Event: Relaxing Evening, Place: Straits Quay, Remarks: Unwind at Straits Quay, a waterfront mall offering a relaxing evening by the sea. Enjoy a leisurely stroll, shop, or dine with a beautiful view., Link: https://maps.app.goo.gl/EzBYV8CQHqdC86dz7}"
    }
  ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Pak Mat Western Cafe, Remarks: Kickstart your day at Pak Mat Western Cafe, where a hearty breakfast awaits. This cozy spot offers a delightful mix of Western flavors, setting the perfect tone for a day of exploration in Penang., Link: https://maps.app.goo.gl/CQL1DLg7X4shKgqJ7} {Event: Lunch, Place: BM Famous Ban Chien Kueh, Remarks: Elevenses at BM Famous Ban Chien Kueh is a must! Indulge in the delightful taste of this local favorite, a perfect mid-morning treat that captures the essence of Penang's culinary traditions., Link: https://maps.app.goo.gl/kTBVVoMQKaL92Gjz7} {Event: Attractions, Place: Pantai Seagate View Pulau Jerjak, Remarks: Explore the breathtaking views at Pantai Seagate on Pulau Jerjak. This hidden gem offers a unique perspective of Penang's coastal beauty, making it a must-visit destination for nature enthusiasts., Link: https://maps.app.goo.gl/a9n8aBvHB3ezAARZ6} {Event: Dinner, Place: Queensbay Mall, Remarks: Queensbay Mall is not just a shopping destination; it's a complete experience. Enjoy a delicious dinner and indulge in some retail therapy, surrounded by a vibrant atmosphere and a plethora of choices., Link: https://maps.app.goo.gl/idTHc6p8gqW2AiTaA} {Event: Attractions, Place: Mengkuang Dam, Remarks: Discover the tranquility of nature at Mengkuang Dam. The serene surroundings and picturesque landscapes make it an ideal destination for those seeking a peaceful escape and scenic beauty., Link: https://maps.app.goo.gl/E3w4QwVDAqQTxT2EA} {Event: Check in hotel, Place: Tropicana 218 Macalister, Remarks: Experience comfort and style at Tropicana 218 Macalister. Your home away from home, this hotel provides a welcoming retreat, ensuring a relaxing stay in the heart of Penang., Link: https://maps.app.goo.gl/Mi3tnvAdj62bJXvL9} {Event: Dessert, Place: Bingsu Monster, Remarks: Satisfy your sweet cravings at Bingsu Monster. This dessert haven offers a cool and refreshing escape with its delightful array of Bingsu creations, a perfect way to beat the Penang heat., Link: https://maps.app.goo.gl/PZB6RouxCKmrxi15A} {Event: Elevenses, Place: Aki Pancake, Remarks: Elevenses take a delightful turn at Aki Pancake. Indulge in a variety of fluffy pancakes with a twist, making it a perfect spot to satisfy your mid-morning cravings., Link: https://maps.app.goo.gl/M8qxkWgBsvZnitRG9}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Pak Mat Western Cafe, Remarks: Kickstart your day at Pak Mat Western Cafe, where a hearty breakfast awaits. This cozy spot offers a delightful mix of Western flavors, setting the perfect tone for a day of exploration in Penang., Link: https://maps.app.goo.gl/CQL1DLg7X4shKgqJ7} {Event: Lunch, Place: BM Famous Ban Chien Kueh, Remarks: Elevenses at BM Famous Ban Chien Kueh is a must! Indulge in the delightful taste of this local favorite, a perfect mid-morning treat that captures the essence of Penang's culinary traditions., Link: https://maps.app.goo.gl/kTBVVoMQKaL92Gjz7} {Event: Elevenses, Place: Aki Pancake, Remarks: Elevenses take a delightful turn at Aki Pancake. Indulge in a variety of fluffy pancakes with a twist, making it a perfect spot to satisfy your mid-morning cravings., Link: https://maps.app.goo.gl/M8qxkWgBsvZnitRG9} {Event: Dinner, Place: Queensbay Mall, Remarks: Queensbay Mall is not just a shopping destination; it's a complete experience. Enjoy a delicious dinner and indulge in some retail therapy, surrounded by a vibrant atmosphere and a plethora of choices., Link: https://maps.app.goo.gl/idTHc6p8gqW2AiTaA} {Event: Attractions, Place: Pantai Seagate View Pulau Jerjak, Remarks: Explore the breathtaking views at Pantai Seagate on Pulau Jerjak. This hidden gem offers a unique perspective of Penang's coastal beauty, making it a must-visit destination for nature enthusiasts., Link: https://maps.app.goo.gl/a9n8aBvHB3ezAARZ6} {Event: Check in hotel, Place: Tropicana 218 Macalister, Remarks: Experience comfort and style at Tropicana 218 Macalister. Your home away from home, this hotel provides a welcoming retreat, ensuring a relaxing stay in the heart of Penang., Link: https://maps.app.goo.gl/Mi3tnvAdj62bJXvL9} {Event: Attractions, Place: Mengkuang Dam, Remarks: Discover the tranquility of nature at Mengkuang Dam. The serene surroundings and picturesque landscapes make it an ideal destination for those seeking a peaceful escape and scenic beauty., Link: https://maps.app.goo.gl/E3w4QwVDAqQTxT2EA} {Event: Dessert, Place: Bingsu Monster, Remarks: Satisfy your sweet cravings at Bingsu Monster. This dessert haven offers a cool and refreshing escape with its delightful array of Bingsu creations, a perfect way to beat the Penang heat., Link: https://maps.app.goo.gl/PZB6RouxCKmrxi15A}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Pak Mat Western Cafe, Remarks: Kickstart your day at Pak Mat Western Cafe, where a hearty breakfast awaits. This cozy spot offers a delightful mix of Western flavors, setting the perfect tone for a day of exploration in Penang., Link: https://maps.app.goo.gl/CQL1DLg7X4shKgqJ7} {Event: Lunch, Place: BM Famous Ban Chien Kueh, Remarks: Elevenses at BM Famous Ban Chien Kueh is a must! Indulge in the delightful taste of this local favorite, a perfect mid-morning treat that captures the essence of Penang's culinary traditions., Link: https://maps.app.goo.gl/kTBVVoMQKaL92Gjz7} {Event: Elevenses, Place: Aki Pancake, Remarks: Elevenses take a delightful turn at Aki Pancake. Indulge in a variety of fluffy pancakes with a twist, making it a perfect spot to satisfy your mid-morning cravings., Link: https://maps.app.goo.gl/M8qxkWgBsvZnitRG9} {Event: Dinner, Place: Queensbay Mall, Remarks: Queensbay Mall is not just a shopping destination; it's a complete experience. Enjoy a delicious dinner and indulge in some retail therapy, surrounded by a vibrant atmosphere and a plethora of choices., Link: https://maps.app.goo.gl/idTHc6p8gqW2AiTaA} {Event: Attractions, Place: Pantai Seagate View Pulau Jerjak, Remarks: Explore the breathtaking views at Pantai Seagate on Pulau Jerjak. This hidden gem offers a unique perspective of Penang's coastal beauty, making it a must-visit destination for nature enthusiasts., Link: https://maps.app.goo.gl/a9n8aBvHB3ezAARZ6} {Event: Check in hotel, Place: Tropicana 218 Macalister, Remarks: Experience comfort and style at Tropicana 218 Macalister. Your home away from home, this hotel provides a welcoming retreat, ensuring a relaxing stay in the heart of Penang., Link: https://maps.app.goo.gl/Mi3tnvAdj62bJXvL9} {Event: Dessert, Place: Bingsu Monster, Remarks: Satisfy your sweet cravings at Bingsu Monster. This dessert haven offers a cool and refreshing escape with its delightful array of Bingsu creations, a perfect way to beat the Penang heat., Link: https://maps.app.goo.gl/PZB6RouxCKmrxi15A} {Event: Attractions, Place: Mengkuang Dam, Remarks: Discover the tranquility of nature at Mengkuang Dam. The serene surroundings and picturesque landscapes make it an ideal destination for those seeking a peaceful escape and scenic beauty., Link: https://maps.app.goo.gl/E3w4QwVDAqQTxT2EA}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Pak Mat Western Cafe, Remarks: Kickstart your day at Pak Mat Western Cafe, where a hearty breakfast awaits. This cozy spot offers a delightful mix of Western flavors, setting the perfect tone for a day of exploration in Penang., Link: https://maps.app.goo.gl/CQL1DLg7X4shKgqJ7} {Event: Lunch, Place: BM Famous Ban Chien Kueh, Remarks: Elevenses at BM Famous Ban Chien Kueh is a must! Indulge in the delightful taste of this local favorite, a perfect mid-morning treat that captures the essence of Penang's culinary traditions., Link: https://maps.app.goo.gl/kTBVVoMQKaL92Gjz7} {Event: Elevenses, Place: Aki Pancake, Remarks: Elevenses take a delightful turn at Aki Pancake. Indulge in a variety of fluffy pancakes with a twist, making it a perfect spot to satisfy your mid-morning cravings., Link: https://maps.app.goo.gl/M8qxkWgBsvZnitRG9} {Event: Dinner, Place: Queensbay Mall, Remarks: Queensbay Mall is not just a shopping destination; it's a complete experience. Enjoy a delicious dinner and indulge in some retail therapy, surrounded by a vibrant atmosphere and a plethora of choices., Link: https://maps.app.goo.gl/idTHc6p8gqW2AiTaA} {Event: Dessert, Place: Bingsu Monster, Remarks: Satisfy your sweet cravings at Bingsu Monster. This dessert haven offers a cool and refreshing escape with its delightful array of Bingsu creations, a perfect way to beat the Penang heat., Link: https://maps.app.goo.gl/PZB6RouxCKmrxi15A} {Event: Check in hotel, Place: Tropicana 218 Macalister, Remarks: Experience comfort and style at Tropicana 218 Macalister. Your home away from home, this hotel provides a welcoming retreat, ensuring a relaxing stay in the heart of Penang., Link: https://maps.app.goo.gl/Mi3tnvAdj62bJXvL9} {Event: Attractions, Place: Mengkuang Dam, Remarks: Discover the tranquility of nature at Mengkuang Dam. The serene surroundings and picturesque landscapes make it an ideal destination for those seeking a peaceful escape and scenic beauty., Link: https://maps.app.goo.gl/E3w4QwVDAqQTxT2EA} {Event: Attractions, Place: Pantai Seagate View Pulau Jerjak, Remarks: Explore the breathtaking views at Pantai Seagate on Pulau Jerjak. This hidden gem offers a unique perspective of Penang's coastal beauty, making it a must-visit destination for nature enthusiasts., Link: https://maps.app.goo.gl/a9n8aBvHB3ezAARZ6}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: George Town Kopitiam, Remarks: Start your day with traditional Penang coffee and kaya toast., Link: https://maps.app.goo.gl/yXURyrfe33ZjnCuQ6} {Event: Attraction, Place: Penang Peranakan Mansion, Remarks: Explore the rich heritage of Peranakan culture., Link: https://maps.app.goo.gl/NsccRCETFuyVAtwE6} {Event: Lunch, Place: Gurney Drive Hawker Centre, Remarks: Enjoy a variety of street food by the beach., Link: https://maps.app.goo.gl/H2B4LXspYcX4g3cw5} {Event: Attraction, Place: Penang Hill, Remarks: Take a funicular ride up for a panoramic view of the island., Link: https://maps.app.goo.gl/6ys4iTdEAnodECRg9} {Event: Dinner, Place: Batu Ferringhi Night Market, Remarks: Experience the vibrant night market and local cuisine., Link: https://maps.app.goo.gl/tdhQ6dR52uMBNQ1E9}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Chulia Street Hawkers, Remarks: Indulge in Penang's famous street food., Link: https://maps.app.goo.gl/bJB8BkRNXTi2tqN86} {Event: Attraction, Place: Kek Lok Si Temple, Remarks: Visit one of the largest Buddhist temples in Southeast Asia., Link: https://maps.app.goo.gl/mTaqSTUawW6N2krb8} {Event: Lunch, Place: Little India, Remarks: Savor authentic Indian cuisine in the heart of Penang., Link: https://maps.app.goo.gl/uY5NWdrWTfNatn2w5} {Event: Attraction, Place: Penang National Park, Remarks: Explore the natural beauty of Penang's national park., Link: https://maps.app.goo.gl/8aSeFS1MQKN6CHJm6} {Event: Dinner, Place: Kimberly Street Food Night Market, Remarks: End your day with a variety of local dishes., Link: https://maps.app.goo.gl/EFc48DVtZUQPFQJU9}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: New Lane Street Foodstalls, Remarks: Experience the morning buzz at a popular local food street., Link: https://maps.app.goo.gl/8fHaZBav4uucx5e9A} {Event: Attraction, Place: Cheong Fatt Tze - The Blue Mansion, Remarks: Discover the history behind the iconic Blue Mansion., Link: https://maps.app.goo.gl/rNDe74FWF3aSX2xS7} {Event: Lunch, Place: Padang Kota Lama, Remarks: Enjoy the sea breeze and local delicacies by the Esplanade., Link: https://maps.app.goo.gl/JqMiwvnBkCiW88yJ6} {Event: Attraction, Place: Penang 3D Trick Art Museum, Remarks: Have fun with interactive 3D art installations., Link: https://maps.app.goo.gl/y2EY8G625nT5ebZw8} {Event: Dinner, Place: Queensbay Mall Food Court, Remarks: Explore modern dining options with a view of the coast., Link: https://maps.app.goo.gl/cm41TzVEzArpM4rZ6}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Joo Hooi Cafe, Remarks: Famous for Penang Assam Laksa and Cendol., Link: https://maps.app.goo.gl/39iDNJKbrEcMaQq79} {Event: Attraction, Place: Wonderfood Museum Penang, Remarks: A fun food-themed museum with giant replicas of Malaysian dishes., Link: https://maps.app.goo.gl/mmgL4EWjixepiE1C6} {Event: Lunch, Place: Kapitan Keling Mosque area, Remarks: Try the street food around one of Penang's most famous mosques., Link: https://maps.app.goo.gl/g63iraxjdabeXzpi6} {Event: Attraction, Place: Penang Time Tunnel, Remarks: Journey through Penang's history with 3D murals and historical exhibits., Link: https://maps.app.goo.gl/KhfS9CYsXC2DQ2gA7} {Event: Dinner, Place: Gurney Paragon Mall, Remarks: A modern mall offering a variety of dining options., Link: https://maps.app.goo.gl/jpHxRgEetwY8Gnss9}"
        }
    ]
},

# 2 Day
{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 2"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Pak Mat Western Cafe, Remarks: Kickstart your day at Pak Mat Western Cafe, where a hearty breakfast awaits. This cozy spot offers a delightful mix of Western flavors, setting the perfect tone for a day of exploration in Penang., Link: https://maps.app.goo.gl/CQL1DLg7X4shKgqJ7} {Event: Lunch, Place: BM Famous Ban Chien Kueh, Remarks: Elevenses at BM Famous Ban Chien Kueh is a must! Indulge in the delightful taste of this local favorite, a perfect mid-morning treat that captures the essence of Penang's culinary traditions., Link: https://maps.app.goo.gl/kTBVVoMQKaL92Gjz7} {Event: Elevenses, Place: Aki Pancake, Remarks: Elevenses take a delightful turn at Aki Pancake. Indulge in a variety of fluffy pancakes with a twist, making it a perfect spot to satisfy your mid-morning cravings., Link: https://maps.app.goo.gl/M8qxkWgBsvZnitRG9} {Event: Dinner, Place: Queensbay Mall, Remarks: Queensbay Mall is not just a shopping destination; it's a complete experience. Enjoy a delicious dinner and indulge in some retail therapy, surrounded by a vibrant atmosphere and a plethora of choices., Link: https://maps.app.goo.gl/idTHc6p8gqW2AiTaA} {Event: Attractions, Place: Pantai Seagate View Pulau Jerjak, Remarks: Explore the breathtaking views at Pantai Seagate on Pulau Jerjak. This hidden gem offers a unique perspective of Penang's coastal beauty, making it a must-visit destination for nature enthusiasts., Link: https://maps.app.goo.gl/a9n8aBvHB3ezAARZ6} {Event: Check in hotel, Place: Tropicana 218 Macalister, Remarks: Experience comfort and style at Tropicana 218 Macalister. Your home away from home, this hotel provides a welcoming retreat, ensuring a relaxing stay in the heart of Penang., Link: https://maps.app.goo.gl/Mi3tnvAdj62bJXvL9} {Event: Attractions, Place: Mengkuang Dam, Remarks: Discover the tranquility of nature at Mengkuang Dam. The serene surroundings and picturesque landscapes make it an ideal destination for those seeking a peaceful escape and scenic beauty., Link: https://maps.app.goo.gl/E3w4QwVDAqQTxT2EA} {Event: Dessert, Place: Bingsu Monster, Remarks: Satisfy your sweet cravings at Bingsu Monster. This dessert haven offers a cool and refreshing escape with its delightful array of Bingsu creations, a perfect way to beat the Penang heat., Link: https://maps.app.goo.gl/PZB6RouxCKmrxi15A}; Day 2: {Event: Breakfast, Place: Pulau Tikus Market Hidden Food Court, Remarks: Embark on a gastronomic adventure at Pulau Tikus Market Hidden Food Court for breakfast. The diverse array of local delicacies will surely tantalize your taste buds, offering a true taste of Penang's culinary richness., Link: https://maps.app.goo.gl/wpRMFpPH1pVMynGt7} {Event: Attractions, Place: Thai Pak Koong Temple Tanjong Tokong, Remarks: Immerse yourself in the cultural charm of Penang at Thai Pak Koong Temple in Tanjong Tokong. The intricate architecture and serene atmosphere make it a must-visit destination for those seeking a glimpse into the rich heritage of the region., Link: https://maps.app.goo.gl/vZAhbsQEknzVMqBGA} {Event: Attractions, Place: Pantai Seagate View Pulau Jerjak, Remarks: Explore the breathtaking views at Pantai Seagate on Pulau Jerjak. This hidden gem offers a unique perspective of Penang's coastal beauty, making it a must-visit destination for nature enthusiasts., Link: https://maps.app.goo.gl/a9n8aBvHB3ezAARZ6} {Event: Lunch, Place: Fuku Eatery & Dessert, Remarks: Indulge in the exquisite flavors of Japan at Fuku Eatery & Dessert for lunch. From savory sushi to delightful desserts, every dish is a masterpiece that transports you to the heart of Japanese cuisine., Link: https://maps.app.goo.gl/78ejCaiD13Ugzoqx8} {Event: Dinner, Place: Pearl Garden Cafe, Remarks: Dine in elegance at Pearl Garden Cafe for dinner. The sophisticated ambiance and delectable menu create a perfect setting for a memorable evening, offering a blend of culinary excellence and refined atmosphere., Link: https://maps.app.goo.gl/djx6jeF9pgLoNSeA7}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 2"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Roti Canai Jalan Transfer, Remarks: Taste the famous Malaysian roti canai at this popular spot., Link: https://maps.app.goo.gl/3DDHGbkoXdJYs31R8} {Event: Attraction, Place: Fort Cornwallis, Remarks: Explore the historical fort with rich colonial heritage., Link: https://maps.app.goo.gl/onMZnEsYxF4aspUu9} {Event: Lunch, Place: Teochew Chendul, Remarks: Enjoy the iconic Penang Chendul dessert., Link: https://maps.app.goo.gl/XX12VAFrcKrJmFS29} {Event: Attraction, Place: The TOP Komtar, Remarks: Visit the tallest building in Penang for breathtaking views., Link: https://maps.app.goo.gl/PWKgVUXDkSVdymvSA} {Event: Dinner, Place: Red Garden Food Paradise & Night Market, Remarks: A vibrant night market offering a plethora of local and international dishes., Link: https://maps.app.goo.gl/kdHZVQDSR2evkGwA6}; Day 2: {Event: Breakfast, Place: Ah Wang Cafe, Remarks: Enjoy a relaxing breakfast with local kaya toast and coffee., Link: https://maps.app.goo.gl/MNEomRrY71VuDQaq8} {Event: Attraction, Place: Penang Avatar Secret Garden, Remarks: Wander through this enchanting, illuminated garden., Link: https://maps.app.goo.gl/sYG7GkrhpfjT4ta28} {Event: Lunch, Place: Nasi Kandar Line Clear, Remarks: Savor the flavors of Penang's famous Nasi Kandar., Link: https://maps.app.goo.gl/VnrMPj7cfXw2f8MQ6} {Event: Attraction, Place: Tropical Spice Garden, Remarks: Discover a variety of spices in a lush, tropical setting., Link: https://maps.app.goo.gl/FNCprbKMxRTyW8N56} {Event: Dinner, Place: Mews Cafe, Remarks: Dine in a cozy environment offering local and fusion cuisine., Link: https://maps.app.goo.gl/Q9eBLsVnJJbcyHPV8}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Penang, day: 2"
        },
        {
            "role": "assistant",
            "content": "Day 1: {Event: Breakfast, Place: Toh Soon Cafe, Remarks: Traditional coffee shop serving great toast and local coffee., Link: https://maps.app.goo.gl/bnk9HwiYPYDHJFMQA} {Event: Attraction, Place: Penang Butterfly Farm, Remarks: Explore the world of butterflies and other insects., Link: https://maps.app.goo.gl/jEdx638PJ5n5uHyMA} {Event: Lunch, Place: Hameediyah Restaurant, Remarks: Experience the taste of Penang's oldest Nasi Kandar., Link: https://maps.app.goo.gl/tGPP121wLHUbQbm56} {Event: Attraction, Place: Made In Penang Interactive Museum, Remarks: A fun, interactive museum showcasing Penang's art and culture., Link: https://maps.app.goo.gl/y2EY8G625nT5ebZw8} {Event: Dinner, Place: Tree Monkey at Tropical Spice Garden, Remarks: Enjoy Thai food amidst the lush greenery of the spice garden., Link: https://maps.app.goo.gl/zXbAqPJabpQsE91b7}; Day 2: {Event: Breakfast, Place: Mugshot Cafe, Remarks: A chic cafe known for its homemade bagels and good coffee., Link: https://maps.app.goo.gl/nGvX8eqUncJuAqyb8} {Event: Attraction, Place: Entopia by Penang Butterfly Farm, Remarks: A nature learning experience with butterflies and various insects., Link: https://maps.app.goo.gl/jEdx638PJ5n5uHyMA} {Event: Lunch, Place: Sri Ananda Bahwan, Remarks: Authentic Indian cuisine in a vibrant setting., Link: https://maps.app.goo.gl/a2nbyaYr9WcFBhLF6} {Event: Attraction, Place: Penang War Museum, Remarks: Visit this historical site that was once a fortress during World War II., Link: https://maps.app.goo.gl/sNYeR2uwiUtaZ8739} {Event: Dinner, Place: China House, Remarks: A unique venue offering a blend of dining, art, and music., Link: https://maps.app.goo.gl/7GYV3gSqaKjFp7k87}"
        }
    ]
},

#KL data
#1 Day

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Kuala Lumpur, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1:{Event: Breakfast, Place: Antipodean Café, Remarks: A cozy spot for a hearty breakfast and great coffee, Link: https://maps.app.goo.gl/AntipodeanCafeKL} {Event: Attraction, Place: Petronas Twin Towers, Remarks: Iconic twin skyscrapers and a symbol of Kuala Lumpur, Link: https://maps.app.goo.gl/PetronasTwinTowers} {Event: Lunch, Place: Jalan Alor Street Food, Remarks: Bustling street lined with local food stalls, Link: https://maps.app.goo.gl/JalanAlorStreetFood} {Event: Attraction, Place: KL Tower, Remarks: Offering panoramic views of the city, Link: https://maps.app.goo.gl/KLTower} {Event: Dinner, Place: Fuego at Troika Sky Dining, Remarks: Contemporary South American cuisine with city views, Link: https://maps.app.goo.gl/FuegoTroikaSkyDining} {Event: Supper, Place: Mamak Jalan Alor, Remarks: Late-night spot for classic Malaysian street food, Link: https://maps.app.goo.gl/MamakJalanAlor}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Kuala Lumpur, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1:{Event: Breakfast, Place: VCR Café, Remarks: Famous for its specialty coffee and cakes, Link: https://maps.app.goo.gl/VCRcafeKL} {Event: Attraction, Place: Batu Caves, Remarks: Limestone hill with a series of caves and temples, Link: https://maps.app.goo.gl/BatuCaves} {Event: Lunch, Place: Lot 10 Hutong Food Court, Remarks: A gourmet heritage village offering diverse cuisines, Link: https://maps.app.goo.gl/Lot10HutongFoodCourt} {Event: Attraction, Place: Islamic Arts Museum Malaysia, Remarks: Exhibits art and artifacts from the Islamic world, Link: https://maps.app.goo.gl/IslamicArtsMuseum} {Event: Dinner, Place: Dining In The Dark KL, Remarks: Unique dining experience in total darkness, Link: https://maps.app.goo.gl/DiningInTheDarkKL} {Event: Supper, Place: Wong Ah Wah, Remarks: Famous for its grilled chicken wings and seafood, Link: https://maps.app.goo.gl/WongAhWah}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Kuala Lumpur, day: 1"
        },
        {
            "role": "assistant",
            "content": "Day 1:{Event: Breakfast, Place: Merchant's Lane, Remarks: A hidden gem with a vintage feel, Link: https://maps.app.goo.gl/MerchantsLane} {Event: Attraction, Place: Sultan Abdul Samad Building, Remarks: A historic building showcasing colonial architecture, Link: https://maps.app.goo.gl/SultanAbdulSamadBuilding} {Event: Lunch, Place: Village Park Restaurant, Remarks: Known for its award-winning nasi lemak, Link: https://maps.app.goo.gl/VillageParkRestaurant} {Event: Attraction, Place: Thean Hou Temple, Remarks: A six-tiered temple of the Chinese sea goddess, Link: https://maps.app.goo.gl/TheanHouTemple} {Event: Dinner, Place: Nobu Kuala Lumpur, Remarks: High-end Japanese-Peruvian fusion cuisine, Link: https://maps.app.goo.gl/NobuKL} {Event: Supper, Place: Blue Boy Vegetarian Food Centre, Remarks: A vegetarian's late-night paradise, Link: https://maps.app.goo.gl/BlueBoyVegetarian}"
        }
    ]
},

#2 Days
{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Kuala Lumpur, day: 2"
        },
        {
            "role": "assistant",
            "content": "Day 1:{Event: Breakfast, Place: Feeka Coffee Roasters, Remarks: Specialty coffees in a relaxing atmosphere, Link: https://maps.app.goo.gl/PHEajXGry5cKEW3p8} {Event: Attraction, Place: Aquaria KLCC, Remarks: An oceanarium showcasing over 5000 different aquatic species, Link: https://maps.app.goo.gl/Xvfcv3p2GrfDRQia6} {Event: Check in, Place: V Hote, Remarks: V Hotel is a trendy and stylish hotel with modern rooms with flat-screen TVs and free Wi-Fi., Link: https://vhotel.com.my/} {Event: Lunch, Place: Madam Kwan's, Remarks: Famous for Malaysian delicacies, Link: https://maps.app.goo.gl/VoNPecqCJ64tYr4L6 }{Event: Attraction, Place: Royal Selangor Visitor Centre, Remarks: Learn about pewter crafting and its history, Link: https://maps.app.goo.gl/j7gj3mcHbADZ8BN1A} {Event: Dinner, Place: Heli Lounge Bar, Remarks: A helicopter pad turned into a dining spot with great views, Link: https://maps.app.goo.gl/wpxH9PegqLPCmedn8} {Event: Supper, Place: Restoran Nasi Kandar Pelita, Remarks: Open 24/7 for authentic Indian-Muslim cuisine, Link: https://maps.app.goo.gl/1rVUBHwbQ6DydQFC6}; Day 2:{Event: Breakfast, Place: Dotty's Pastries & Coffee, Remarks: Famous for its salted egg yolk cronut, Link: https://maps.app.goo.gl/qpXHsZMN6aq8WfoBA}{Event: Attraction, Place: Menara Kuala Lumpur, Remarks: The 7th tallest freestanding tower in the world, Link: https://maps.app.goo.gl/PeBoWiV47cG8s75k8} {Event: Lunch, Place: Little Penang Kafe, Remarks: A taste of Penang in the heart of KL, Link: https://maps.app.goo.gl/Pq3U3Pp9XHe9Vj149} {Event: Attraction, Place: Central Market Kuala Lumpur, Remarks: A cultural landmark for arts, crafts, and souvenirs, Link: https://maps.app.goo.gl/GFGsJdP5VRpcwrD29} {Event: Dinner, Place: Marini's on 57, Remarks: Italian fine dining with stunning views of the city skyline, Link: https://maps.app.goo.gl/V4T8m6obn26KhTRS6} {Event: Supper, Place: Restoran Kin Kin, Remarks: Famous for its chili pan mee, Link: https://maps.app.goo.gl/sneiR91dqQXd67VGA}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Kuala Lumpur, day: 2"
        },
        {
            "role": "assistant",
            "content": "Day 1:{Event: Breakfast, Place: The Toast Co., Remarks: Serves a variety of toasts and brunch items, Link: https://maps.app.goo.gl/yxw3sgHigKHiQAgi} {Event: Attraction, Place: Perdana Botanical Garden, Remarks: A lush green escape in the city center, Link: https://maps.app.goo.gl/oHugF7USwqWcQgwt6} {Event: Lunch, Place: Nasi Lemak Bumbung Seapark, Remarks: Famous for its nasi lemak served under the stars, Link: https://maps.app.goo.gl/mjJrpgWtKXGNeaHt6} {Event: Check in, Place: De Elements Business Hotel KL, Remarks: De Element business Hotel offers a wide range of rooms and suites which are fully equipped with modern facilities to accommodate your stay., Link: https://www.deelementshotel.com/} {Event: Attraction, Place: KL Tower, Remarks: Offering panoramic views of the city, Link: https://maps.app.goo.gl/NYeFymbCqFpg58Cg6} {Event: Dinner, Place: Fuego at Troika Sky Dining, Remarks: Contemporary South American cuisine with city views, Link: https://maps.app.goo.gl/jFF2tFJGAMT63aSz8} {Event: Supper, Place: Wong Ah Wah, Remarks: Famous for its grilled chicken wings and seafood, Link: https://maps.app.goo.gl/jPkGb77X8VrQn4sB6}; Day 2:{Event: Breakfast, Place: VCR Café, Remarks: Famous for its specialty coffee and cakes, Link: https://maps.app.goo.gl/PrS4CNeeQqJB5sA69} {Event: Attraction, Place: Islamic Arts Museum Malaysia, Remarks: Exhibits art and artifacts from the Islamic world, Link: https://maps.app.goo.gl/mnpDWHM11zXCNr7f7} {Event: Lunch, Place: Village Park Restaurant, Remarks: Known for its award-winning nasi lemak, Link: https://maps.app.goo.gl/fzRKW4kQpuJfgVtz7} {Event: Attraction, Place: Thean Hou Temple, Remarks: A six-tiered temple of the Chinese sea goddess, Link: https://maps.app.goo.gl/AqywmrC7evSxCjBf8} {Event: Dinner, Place: Nobu Kuala Lumpur, Remarks: High-end Japanese-Peruvian fusion cuisine, Link: https://maps.app.goo.gl/HH7YDBrTW5CWWMJL6} {Event: Supper, Place: Kedai Mamak Husin, Remarks: Late-night spot for classic Malaysian street food, Link: https://maps.app.goo.gl/fowwPoicZp2uLhuU6}"
        }
    ]
},

#3 Days
{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Kuala Lumpur, day: 3"
        },
        {
            "role": "assistant",
            "content": "Day 1:{Event: Breakfast, Place: The Toast Co., Remarks: Serves a variety of toasts and brunch items, Link: https://maps.app.goo.gl/SGN4bRpCRCYW4nwq7} {Event: Attraction, Place: Perdana Botanical Garden, Remarks: A lush green escape in the city center, Link: https://maps.app.goo.gl/oHugF7USwqWcQgwt6} {Event: Lunch, Place: Nasi Lemak Bumbung Seapark, Remarks: Famous for its nasi lemak served under the stars, Link: https://maps.app.goo.gl/mjJrpgWtKXGNeaHt6} {Event: Check in, Place: Regalia Upper View Hotel, Remarks: Sunway Putra Mall and Putra World Trade Centre (PWTC) is just 4 minutes’ walk away while the Open University is nearby., Link: https://www.upperview-regalia.com/} {Event: Attraction, Place: Central Market Kuala Lumpur, Remarks: A cultural landmark for arts, crafts, and souvenirs, Link: https://maps.app.goo.gl/bjpeV8nfEo6MvAK48} {Event: Dinner, Place: Marini's on 57, Remarks: Italian fine dining with stunning views of the city skyline, Link: https://maps.app.goo.gl/V4T8m6obn26KhTRS6} {Event: Supper, Place: Wong Ah Wah, Remarks: Famous for its grilled chicken wings and seafood, Link: https://maps.app.goo.gl/jPkGb77X8VrQn4sB6}; Day 2:{Event: Breakfast, Place: Merchant's Lane, Remarks: A hidden gem with a vintage feel, Link: https://maps.app.goo.gl/jYAHJNZBhFJvdNgu5} {Event: Attraction, Place: Batu Caves, Remarks: Limestone hill with a series of caves and temples, Link: https://maps.app.goo.gl/6ydWimBqdZvgRvQt7} {Event: Lunch, Place: Jalan Alor Street Food, Remarks: Bustling street lined with local food stalls, Link: https://maps.app.goo.gl/5qij3bK7khrx6xUq8} {Event: Attraction, Place: Sultan Abdul Samad Building, Remarks: A historic building showcasing colonial architecture, Link: https://maps.app.goo.gl/etoHJ4QBoHzA2QZh6 } {Event: Dinner, Place: Dining In The Dark KL, Remarks: Unique dining experience in total darkness, Link: https://maps.app.goo.gl/2rxeQ9SiZ3xib4Yq9} {Event: Supper, Place: Blue Boy Vegetarian Food Centre, Remarks: A vegetarian's late-night paradise, Link: https://maps.app.goo.gl/vAtPUQsUE5WxBejw9}; Day 3:{Event: Breakfast, Place: Inside Scoop, Remarks: Delicious homemade ice creams and waffles, Link: https://maps.app.goo.gl/kBM8TqVSfY624Sia7} {Event: Attraction, Place: Islamic Arts Museum Malaysia, Remarks: Exhibits art and artifacts from the Islamic world, Link: https://maps.app.goo.gl/mnpDWHM11zXCNr7f7} {Event: Lunch, Place: KAKIGŌRI, Remarks: Japanese shaved ice dessert parlor, Link: https://maps.app.goo.gl/xuEDrTjDV9FB74WL6} {Event: Attraction, Place: KL Tower, Remarks: Offering panoramic views of the city, Link: https://maps.app.goo.gl/NYeFymbCqFpg58Cg6} {Event: Dinner, Place: Whimsical Gelateria & Caffe, Remarks: Known for its nitrogen ice cream and unique flavors, Link: https://maps.app.goo.gl/4r9yxbczyu1Soney6} {Event: Supper, Place: MYTH Espresso Bar, Remarks: Specialty coffee bar with a selection of pastries, Link: https://maps.app.goo.gl/Z6XPv25ieAiiYcoo7}"
        }
    ]
},

{
    "messages": [
        {
            "role": "system",
            "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."
        },
        {
            "role": "user",
            "content": "Plan an itinerary based on following condition, country: Malaysia, state: Kuala Lumpur, day: 3"
        },
        {
            "role": "assistant",
            "content": "Day 1:{Event: Breakfast, Place: Merchant's Lane, Remarks: A hidden gem with a vintage feel, Link: https://maps.app.goo.gl/jYAHJNZBhFJvdNgu5} {Event: Attraction, Place: Batu Caves, Remarks: Limestone hill with a series of caves and temples, Link: https://maps.app.goo.gl/Bwe6jGyBkYfTHNqt6} {Event: Lunch, Place: Jalan Alor Street Food, Remarks: Bustling street lined with local food stalls, Link: https://maps.app.goo.gl/BR3jDPQfN3kqcGa28} {Event: Attraction, Place: Sultan Abdul Samad Building, Remarks: A historic building showcasing colonial architecture, Link: https://maps.app.goo.gl/etoHJ4QBoHzA2QZh6} {Event: Check in, Place: Dorsett Kuala Lumpur, Remarks: The 4-star Ascott Kuala Lumpur Aparthotel lies in Kuala Lumpur City-Centre district within a walking distance to Petronas Twin Towers. Situated only 0.4 km from Suria KLCC Shopping Mall, this elegant venue comes with fitness and the gym., Link: https://dorsett-kuala-lumpur.hotels-in-kuala-lumpur.com/en/} {Event: Dinner, Place: Dining In The Dark KL, Remarks: Unique dining experience in total darkness, Link: https://maps.app.goo.gl/2rxeQ9SiZ3xib4Yq9} {Event: Supper, Place: Blue Boy Vegetarian Food Centre, Remarks: A vegetarian's late-night paradise, Link: https://maps.app.goo.gl/vAtPUQsUE5WxBejw9}; Day 2:{Event: Breakfast, Place: Antipodean Café, Remarks: A cozy spot for a hearty breakfast and great coffee, Link: https://maps.app.goo.gl/derLQBDKkrysRjQJ7} {Event: Attraction, Place: Petronas Twin Towers, Remarks: Iconic twin skyscrapers and a symbol of Kuala Lumpur, Link: https://maps.app.goo.gl/n5BsfVvfSPwSds6o7} {Event: Lunch, Place: Lot 10 Hutong Food Court, Remarks: A gourmet heritage village offering diverse cuisines, Link: https://maps.app.goo.gl/tagytu44SbjsBauHA} {Event: Attraction, Place: National Mosque of Malaysia, Remarks: A modern architectural marvel representing Islamic art, Link: https://maps.app.goo.gl/6DBrtwtKi6J28bU56} {Event: Dinner, Place: Opium KL, Remarks: Asian cuisine with a twist in a unique setting, Link: https://maps.app.goo.gl/UD5zoCB64XVovMHf7} {Event: Supper, Place: SS2 Mamak, Remarks: A popular spot for late-night roti canai and teh tarik, Link: https://maps.app.goo.gl/WhMGhbSQG4xJFRN86}; Day 3:{Event: Breakfast, Place: Dotty's Pastries & Coffee, Remarks: Famous for its salted egg yolk cronut, Link: https://maps.app.goo.gl/qpXHsZMN6aq8WfoBA} {Event: Attraction, Place: Aquaria KLCC, Remarks: An oceanarium showcasing over 5000 different aquatic species, Link: https://maps.app.goo.gl/bXSoZLifQY1xJrRc7} {Event: Lunch, Place: Madam Kwan's, Remarks: Famous for Malaysian delicacies, Link: https://maps.app.goo.gl/6sVHTZQDVrikG5uD9} {Event: Attraction, Place: Menara Kuala Lumpur, Remarks: The 7th tallest freestanding tower in the world, Link: https://maps.app.goo.gl/PeBoWiV47cG8s75k8} {Event: Dinner, Place: Heli Lounge Bar, Remarks: A helicopter pad turned into a dining spot with great views, Link: https://maps.app.goo.gl/epVYHq5qAR2p3PFZ8} {Event: Supper, Place: Restoran Kin Kin, Remarks: Famous for its chili pan mee, Link: https://maps.app.goo.gl/safzPWbFf6NKPy3X7}"
        }
    ]
}


]
    

print(data) 

In [30]:
#Converting the database to a suitable format\
import jsonlines

jsonl_filename = "output.jsonl"

#save as JSONL (Json line file)
with jsonlines.open(jsonl_filename,"w") as writer:
    writer.write_all(data)


In [ ]:
file = client.files.create(
  file = open("output.jsonl", "rb"),
  purpose = "fine-tune"
)
 
 
# Extracting the dataset id
dataset = file.id
print(dataset)

**Training our model**

In [ ]:
# Starting a fine-tuning job
trained_model = client.fine_tuning.jobs.create(
  training_file=dataset, 
  model="gpt-3.5-turbo",
hyperparameters={
    "n_epochs":15
  }
)

# Retrieving The model Training Status
jobstatus = client.fine_tuning.jobs.retrieve(trained_model.id)
print(jobstatus)

In [ ]:
# Retrieving The model Training Status
jobstatus = client.fine_tuning.jobs.retrieve(trained_model.id)
print(jobstatus)

**Extracting Model**

In [ ]:
# Retrieving The fine-tuned model id
fine_tuned_model = jobstatus.fine_tuned_model
print(fine_tuned_model)

Interacting with ai


In [57]:
# Generate the response
def answer(question):

    prompt = [{"role": "system", "content": "WanderBot is an advanced trip planner AI designed to crafting seamless itineraries."}, {"role": "user", "content": question}]
    response = client.chat.completions.create(
        model=fine_tuned_model,
        messages=prompt 
        )
    
    response_msg = response.choices[0].message.content

    return response_msg

In [ ]:
#prompt for input from user
country = input("Country: ")
state = input("State: ")
day = input("Number of day: ")

command = f"Plan an itinerary based on following coondition, country: {country}, state: {state}, day: {day}"

# Generate the response
output = answer(command)
print(output)

In [ ]:
import re

# Extracting individual events using regular expression
events = re.findall(r'\{([^}]+)\}', output)

# Print table header
print("{:<10} {:<25} {:<100} {:<50}".format("Event", "Place", "Remarks", "Link"))
print("="*185)  # Print a line to separate header from content

# Print table content
for event in events:
    # Extracting key-value pairs using regular expression
    key_value_pairs = re.findall(r'(\w+):\s*([^,]+),?', event)
    event_dict = dict(key_value_pairs)

    print("{:<10} {:<25} {:<100} {:<50}".format(event_dict.get("Event", ""),
                                                 event_dict.get("Place", ""),
                                                 event_dict.get("Remarks", ""),
                                                 event_dict.get("Link", "")))

